# **8 brands** - YSL, Prada, Gucci, Hermès, LV, Chanel, Fendi, Balenciaga

In [ ]:
from google.colab import drive

drive.mount('/content/grive')  

Mounted at /content/grive


In [ ]:
!nvidia-smi

Thu Aug 19 22:50:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import os #shutil
import os.path
import pathlib
from pathlib import Path
from glob import glob


import PIL
import PIL.Image
#import tensorflow as tf


import cv2
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
os.chdir("/content/grive/My Drive/grive/farfetch")
!ls

data_dir = '/content/grive/My Drive/grive/farfetch/bag_8'

bag_  bag_8  bag_8_img_aug  bag_img_aug  ff_bag_8_v1.h5


In [ ]:
os.getcwd()

'/content/grive/My Drive/grive/farfetch'

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255,
                              validation_split=0.2) 

In [ ]:
batch_size = 32  
img_height = 150  
img_width = 150 

In [ ]:
train_generator = image_gen.flow_from_directory(data_dir,
                                          target_size=(img_height, img_width),
                                          batch_size=batch_size,
                                          seed=42,
                                          shuffle=False,
                                          subset='training',           # train_val_split
                                          class_mode='categorical')

Found 2266 images belonging to 8 classes.


In [ ]:
val_generator = image_gen.flow_from_directory(data_dir,
                                        target_size=(img_height, img_width),
                                        batch_size=batch_size,
                                        seed=42,
                                        shuffle=False,
                                        subset='validation',
                                        class_mode='categorical')

Found 563 images belonging to 8 classes.


In [ ]:
train_generator.class_indices

{'bag_Balenciaga_': 0,
 'bag_Chanel_': 1,
 'bag_Fendi_': 2,
 'bag_Gucci_': 3,
 'bag_Hermes_': 4,
 'bag_LV_': 5,
 'bag_Prada_': 6,
 'bag_YSL_': 7}

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, InputLayer, Dropout
from keras.applications.vgg16 import VGG16   


In [ ]:
# transfer learning

base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))   

# Freeze convolutional layers
for layer in base.layers:
    layer.trainable = False  

model8_v1 = Sequential(
                        [InputLayer(input_shape=(150,150,3)),    
                         base,
                         Flatten(),    
                         #Dense(1000, activation='relu'),  # add 1 more layer
                         Dense(1000, activation='relu'),   
                         Dense(1000, activation='relu'),   # add Dropout(0.05)             
                         Dense(8, activation='softmax')]
                       )

model8_v1.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
result8_v1 = model8_v1.fit_generator(train_generator, 
                                            validation_data=val_generator, 
                                            epochs=20,  # 10
                                            verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
71/71 [==============================] - 543s 7s/step - loss: 3.0137 - accuracy: 0.1995 - val_loss: 1.9574 - val_accuracy: 0.2806
Epoch 2/20
71/71 [==============================] - 18s 258ms/step - loss: 2.0285 - accuracy: 0.2176 - val_loss: 1.9168 - val_accuracy: 0.3055
Epoch 3/20
71/71 [==============================] - 19s 263ms/step - loss: 1.9261 - accuracy: 0.2440 - val_loss: 1.7270 - val_accuracy: 0.3233
Epoch 4/20
71/71 [==============================] - 19s 265ms/step - loss: 1.7252 - accuracy: 0.3553 - val_loss: 1.8555 - val_accuracy: 0.3517
Epoch 5/20
71/71 [==============================] - 19s 262ms/step - loss: 1.6511 - accuracy: 0.3778 - val_loss: 1.5793 - val_accuracy: 0.4227
Epoch 6/20
71/71 [==============================] - 18s 260ms/step - loss: 1.6122 - accuracy: 0.3910 - val_loss: 1.5586 - val_accuracy: 0.4121
Epoch 7/20
71/71 [==============================] - 19s 262ms/step - loss: 1.4453 - accuracy: 0.4629 - val_loss: 1.6887 - val_accuracy: 0.4103
E

In [ ]:
model8_v1.save('ff_bag_8_v1.h5')

In [ ]:
# add callback

result8_vc1 = model8_v1.fit_generator(                
        train_generator, validation_data=val_generator , epochs=100,  verbose=1,    
        callbacks=[
            keras.callbacks.EarlyStopping(patience=4, verbose=1, restore_best_weights=True),   
            keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1),              
        ])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
71/71 [==============================] - 18s 253ms/step - loss: 0.8057 - accuracy: 0.7207 - val_loss: 1.0843 - val_accuracy: 0.6607
Epoch 2/100
71/71 [==============================] - 17s 242ms/step - loss: 0.6475 - accuracy: 0.7877 - val_loss: 1.0295 - val_accuracy: 0.6643
Epoch 3/100
71/71 [==============================] - 17s 241ms/step - loss: 0.5786 - accuracy: 0.8049 - val_loss: 1.3555 - val_accuracy: 0.5790
Epoch 4/100
71/71 [==============================] - 17s 245ms/step - loss: 0.6757 - accuracy: 0.7520 - val_loss: 1.7101 - val_accuracy: 0.5346
Epoch 5/100
71/71 [==============================] - 17s 244ms/step - loss: 0.8415 - accuracy: 0.7008 - val_loss: 1.0813 - val_accuracy: 0.6661

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/100
71/71 [==============================] - 17s 242ms/step - loss: 0.5218 - accuracy: 0.8305 - val_loss: 1.1258 - val_accuracy: 0.6767
Restoring model weights from the end of the best epoch.

In [ ]:
model8_v1.save('ff_bag_8_v2.h5')

In [ ]:
# image augmentation

train_datagen = ImageDataGenerator(
      rescale=1./255,  
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir = '/content/grive/My Drive/grive/farfetch/bag_8_img_aug/train'         
val_dir = '/content/grive/My Drive/grive/farfetch/bag_8_img_aug/test'

In [ ]:
train_generator_aug = train_datagen.flow_from_directory( 
        train_dir,  
        target_size=(150, 150),
        batch_size=20,  #32
        class_mode = 'categorical')


val_generator_aug = val_datagen.flow_from_directory(
        val_dir, 
        target_size=(150, 150),
        batch_size=20,  #32
        class_mode = 'categorical')

Found 2260 images belonging to 8 classes.
Found 569 images belonging to 8 classes.


In [ ]:
result_aug = model8_v1.fit_generator(train_generator_aug, 
                         validation_data=val_generator_aug , 
                         epochs=10, 
                         verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
113/113 [==============================] - 472s 4s/step - loss: 1.6324 - accuracy: 0.4310 - val_loss: 0.7772 - val_accuracy: 0.7522
Epoch 2/10
113/113 [==============================] - 29s 253ms/step - loss: 1.4690 - accuracy: 0.4819 - val_loss: 0.7645 - val_accuracy: 0.7540
Epoch 3/10
113/113 [==============================] - 29s 253ms/step - loss: 1.3760 - accuracy: 0.5190 - val_loss: 0.7303 - val_accuracy: 0.7540
Epoch 4/10
113/113 [==============================] - 28s 252ms/step - loss: 1.3111 - accuracy: 0.5398 - val_loss: 0.7281 - val_accuracy: 0.7504
Epoch 5/10
113/113 [==============================] - 28s 251ms/step - loss: 1.2430 - accuracy: 0.5681 - val_loss: 0.6547 - val_accuracy: 0.7786
Epoch 6/10
113/113 [==============================] - 28s 251ms/step - loss: 1.2404 - accuracy: 0.5668 - val_loss: 0.7226 - val_accuracy: 0.7663
Epoch 7/10
113/113 [==============================] - 28s 250ms/step - loss: 1.2440 - accuracy: 0.5593 - val_loss: 0.6988 - val_accu

In [ ]:
# add callback 

result_aug_call = model8_v1.fit_generator(         # class_weight=None,       
        train_generator_aug, validation_data=val_generator_aug , epochs=100,  verbose=1,  
        callbacks=[
            keras.callbacks.EarlyStopping(patience=4, verbose=1, restore_best_weights=True),  
            keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1),             
        ])




/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
113/113 [==============================] - 31s 275ms/step - loss: 1.1187 - accuracy: 0.6102 - val_loss: 0.6501 - val_accuracy: 0.7715
Epoch 2/100
113/113 [==============================] - 32s 287ms/step - loss: 1.0850 - accuracy: 0.6164 - val_loss: 0.6860 - val_accuracy: 0.7592
Epoch 3/100
113/113 [==============================] - 31s 274ms/step - loss: 1.0777 - accuracy: 0.6314 - val_loss: 0.6796 - val_accuracy: 0.7750
Epoch 4/100
113/113 [==============================] - 30s 265ms/step - loss: 1.1222 - accuracy: 0.6137 - val_loss: 0.6636 - val_accuracy: 0.7768

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 5/100
113/113 [==============================] - 30s 265ms/step - loss: 1.0253 - accuracy: 0.6447 - val_loss: 0.6577 - val_accuracy: 0.7838
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping


In [ ]:
model8_v1.save('ff_bag_8_v3.h5')

In [ ]:
# add dropout
model8_v1.add(Dense(512, activation='relu'))
model8_v1.add(Dropout(.05))   #
model8_v1.add(Dense(8, activation='softmax')) 

result_aug_5 = model8_v1.fit_generator(train_generator_aug, 
                         validation_data=val_generator_aug , 
                         epochs=10, 
                         verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
113/113 [==============================] - 30s 270ms/step - loss: 1.1009 - accuracy: 0.6190 - val_loss: 0.6349 - val_accuracy: 0.7750
Epoch 2/10
113/113 [==============================] - 29s 256ms/step - loss: 1.0623 - accuracy: 0.6301 - val_loss: 0.6428 - val_accuracy: 0.7750
Epoch 3/10
113/113 [==============================] - 29s 254ms/step - loss: 1.0427 - accuracy: 0.6363 - val_loss: 0.6622 - val_accuracy: 0.7557
Epoch 4/10
113/113 [==============================] - 31s 276ms/step - loss: 1.0422 - accuracy: 0.6376 - val_loss: 0.6388 - val_accuracy: 0.7821
Epoch 5/10
113/113 [==============================] - 31s 273ms/step - loss: 1.0293 - accuracy: 0.6350 - val_loss: 0.6350 - val_accuracy: 0.7873
Epoch 6/10
113/113 [==============================] - 29s 260ms/step - loss: 1.0281 - accuracy: 0.6403 - val_loss: 0.6529 - val_accuracy: 0.7768
Epoch 7/10
113/113 [==============================] - 29s 255ms/step - loss: 1.0029 - accuracy: 0.6558 - val_loss: 0.6313 - val_ac

In [ ]:
# add dropout more

model8_v1.add(Dense(512, activation='relu'))
model8_v1.add(Dropout(.1))   #
model8_v1.add(Dense(6, activation='softmax')) 

result_aug_6 = model8_v1.fit_generator(train_generator_aug, 
                         validation_data=val_generator_aug , 
                         epochs=10, 
                         verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
113/113 [==============================] - 30s 267ms/step - loss: 0.9829 - accuracy: 0.6575 - val_loss: 0.6538 - val_accuracy: 0.7733
Epoch 2/10
113/113 [==============================] - 31s 279ms/step - loss: 0.9837 - accuracy: 0.6588 - val_loss: 0.6355 - val_accuracy: 0.7838
Epoch 3/10
113/113 [==============================] - 31s 277ms/step - loss: 0.9678 - accuracy: 0.6549 - val_loss: 0.6455 - val_accuracy: 0.7768
Epoch 4/10
113/113 [==============================] - 31s 277ms/step - loss: 0.9529 - accuracy: 0.6695 - val_loss: 0.6298 - val_accuracy: 0.7838
Epoch 5/10
113/113 [==============================] - 32s 281ms/step - loss: 0.9823 - accuracy: 0.6588 - val_loss: 0.6348 - val_accuracy: 0.7768
Epoch 6/10
113/113 [==============================] - 32s 280ms/step - loss: 0.9714 - accuracy: 0.6584 - val_loss: 0.6342 - val_accuracy: 0.7803
Epoch 7/10
113/113 [==============================] - 31s 276ms/step - loss: 0.9177 - accuracy: 0.6810 - val_loss: 0.6321 - val_ac

In [ ]:
model8_v1.save('ff_bag_8_v4.h5')

In [ ]:
# best model 

# 1. metric

# change RESULT

acc = result_NN_t.history['accuracy']
val_acc = result_NN_t.history['val_accuracy']

loss = result_NN_t.history['loss']
val_loss = result_NN_t.history['val_loss']


epochs_range = range(epochs)


plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy') #, color = 'red')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')  # , color='blue')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')


plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
# 2. confusion matrix
from sklearn.metrics import confusion_matrix

In [ ]:
# 3. Classification report